<a href="https://colab.research.google.com/github/pierresegonne/ELEC5307_Project2/blob/master/Task2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# UTILS

In [0]:
# Install pytorch

from os import path
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())

accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'
print('Accelerator: ', accelerator)

print('Installing Torch') 
!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-1.0.0-{platform}-linux_x86_64.whl
# Issue with torchvision version
!pip install torchvision==0.2.1
import torch
import torchvision

Accelerator:  cu80
Installing Torch
     |████████████████████████████████| 532.5MB 1.2MB/s 
ERROR: torchvision 0.4.1+cu100 has requirement torch==1.3.0, but you'll have torch 1.0.0 which is incompatible.
     |████████████████████████████████| 61kB 1.9MB/s 
  Found existing installation: torchvision 0.4.1+cu100
    Uninstalling torchvision-0.4.1+cu100:
      Successfully uninstalled torchvision-0.4.1+cu100


In [0]:
ROOT_DIRECTORY = '.'

In [0]:
# Allow saving files to google drive if ran on colab
from google.colab import drive
drive.mount('/content/gdrive')
ROOT_DIRECTORY = '/content/gdrive/My Drive/Colab Notebooks/ELEC5307 Project2'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


# IMPORTS

In [0]:
# useful packages
import torch
import torchvision
import torchvision.transforms as transforms
from torch import nn

import torch.nn.functional as F
from torch.utils.data import SubsetRandomSampler
import torch.optim as optim


import numpy as np
import math
import random

import matplotlib.pyplot as plt

import os
import pickle

# HYPERPARAMETERS

# DATA LOADING AND TRANSFORMATION

# MODEL STRUCTURE



*   https://github.com/pytorch/vision/blob/master/torchvision/models/alexnet.py
*   https://github.com/wogong/pytorch-dann



In [0]:
class Reverse(torch.autograd.Function):

  @staticmethod
  def forward(ctx, x, lbd):
    ctx.lbd = lbd
    return x.view_as(x)
  
  @staticmethod
  def backward(ctx, grad_output):
    output = grad_output.neg() * ctx.lbd
    return output, None


class DANN(nn.Module):

  def __init__(self, number_classes=10):
    # TODO
    # Can we pass a list to nn.sequential?

    model_alexnet = torchvision.models.alexnet(pretrained=True)

    self.features = model_alexnet.features

    self.fc = nn.Sequential()

    self.class_classifier = nn.Sequential(
        nn.Linear(?, number_classes)
    )

    self.domain_classifier = nn.Sequential(
        
    )

  def forward(self, input, lbd):
    features = self.features(input)
    features = features.view(???)
    fc = self.fc(features)
    bottleneck = self.bottleneck(fc)
    reverse_bottleneck = Reverse.apply(bottleneck, lbd)

    predicted_classes = self.class_classifier(bottleneck)
    predicted_domains = self.domain_classifier(reverse_bottleneck)

    return predicted_classes, predicted_domains